<a href="https://colab.research.google.com/github/DonRomaniello/CitibikeDocks/blob/master/TripData.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Note: While CitiBike has stations on both sides of the Hudson, few (if any) rides originate in one state and end in another. There would be very little incentive to attempt this feat beyond bragging rights, and based on the two sets of trip data published depending on jurisdiction, it does not seem like anyone is doing it. As I live and work in New York City, I will only be focusing on New York.

Before training our model, it would be useful to learn how bikes flow between stations.

CitiBike publishes trip reports every month to an AWS S3 bucket. These reports contain data of all the trips taken 
by CitiBike users, with information like the start times and locations, end times and locations, etc.

In [110]:
import requests
import pandas as pd

Unfortunately, some of the data are published as zip files that also contain MacOS special files, which means PANDAS can't simply ingest the zip file as published.

We will use Requests to grab the file from the S3 bucket, BytesIO to keep the zip directory in memory, and ZipFile to work with the zip directory to extract the CSV only.

In [111]:
from io import BytesIO
from zipfile import ZipFile

In [112]:
dirtyZipUrl = 'https://s3.amazonaws.com/tripdata/202108-citibike-tripdata.csv.zip'
dirtyZipFilename = requests.get(dirtyZipUrl).content
dirtyZipFile = ZipFile( BytesIO(dirtyZipFilename), 'r')

for item in dirtyZipFile.namelist():
  print("File in zip:" + item)

File in zip:202108-citibike-tripdata.csv
File in zip:__MACOSX/._202108-citibike-tripdata.csv


There it is, the stuff that PANDAS doesn't like. The files in the "__MACOSX" directory will cause the PANDAS read_csv() function to throw an exception.

Not all of the published zip directories have this problem, but we shoud get rid of it if it is in there.


In [113]:
justCSV = [cleanFilename for cleanFilename in dirtyZipFile.namelist() if "._" not in cleanFilename and ".csv" in cleanFilename][0]

And now we can load the data and make sure it is as expected.

In [114]:
tripData = pd.read_csv(dirtyZipFile.open(justCSV), low_memory=False)
tripData.head()

,ride_id,rideable_type,started_at,ended_at,start_station_name,start_station_id,end_station_name,end_station_id,start_lat,start_lng,end_lat,end_lng,member_casual
0,FB6B89D05B67EBED,classic_bike,2021-08-24 15:59:57,2021-08-24 16:42:07,Broadway & E 21 St,6098.10,Central Park North & Adam Clayton Powell Blvd,7617.07,40.739888,-73.989586,40.799484,-73.955613,member
1,E13DA3E30CEF8DFC,classic_bike,2021-08-18 13:12:01,2021-08-18 13:21:26,E 13 St & 2 Ave,5820.08,Henry St & Grand St,5294.04,40.731539,-73.985302,40.714211,-73.981095,member
2,56617490AB8AE69C,classic_bike,2021-08-17 14:31:23,2021-08-17 14:35:34,E 95 St & 3 Ave,7365.13,E 84 St & Park Ave,7243.04,40.784903,-73.950503,40.778627,-73.957721,member
3,CA908B271C7D6663,classic_bike,2021-08-11 10:00:12,2021-08-11 10:31:01,Madison Ave & E 82 St,7188.13,E 84 St & Park Ave,7243.04,40.778131,-73.960694,40.778627,-73.957721,casual
4,3E170CE1F4FE179D,classic_bike,2021-08-12 19:28:38,2021-08-12 19:48:50,E 74 St & 1 Ave,6953.08,E 84 St & Park Ave,7243.04,40.768974,-73.954823,40.778627,-73.957721,casual


Great.

We should turn this process into a function that takes the URL of the S3 item as input and returns a pandas DataFrame, because we will be doing this many times.

In [115]:
def readDirtyZip(dirtyZipUrl):
  dirtyZipFilename = requests.get(dirtyZipUrl).content
  dirtyZipFile = ZipFile( BytesIO(dirtyZipFilename), 'r')
  tripData = pd.read_csv(dirtyZipFile.open([cleanFilename for cleanFilename in dirtyZipFile.namelist() if "._" not in cleanFilename and ".csv" in cleanFilename][0]), low_memory=False)
  
  return tripData

# Legacy Data

Before going any further in creating our trip dataset, there is a slight wrinkle. At some point CitiBike changed the IDs for the all the stations. 

Graciously, they saw fit to include the old names *and* new names in the JSON feed that provides live information about the system.

This will allow us to construct a dictionary that which we can use to rename the old trip data to reflect the current naming paradigm.

Notes:  

*   Stations that begin with letters include stations in New Jersey, so we will remove them when we make the dictionary.
* The legacy system used int64 as the datatype for station IDs. The new system uses strings. When constructing the dictionary, the legacy IDs need to be type cast.


In [116]:
stationLocationsRequest = requests.get('https://gbfs.citibikenyc.com/gbfs/en/station_information.json')
stationLocationData = stationLocationsRequest.json()
stationLocations = pd.DataFrame(stationLocationData['data']['stations'])
stationNameDictionary = dict(zip(stationLocations[stationLocations['short_name'].str.contains('[a-zA-Z]+', regex=True)==False].legacy_id.astype('int64'), stationLocations[stationLocations['short_name'].str.contains('[a-zA-Z]+', regex=True)==False].short_name))

We don't need anything except the dictionary, so we will delete everything else that went into creating the dictionary.

In [117]:
del stationLocationsRequest, stationLocationData, stationLocations

Since we are trying to predict the availability of bikes and open docks in the current CitiBike system, the new names will be used to rename old trip station IDs.

The last month that used the legacy IDs appears to be January, 2021. We should test our renaming dictionary on this before proceeding.

In [118]:
legacyTrips = readDirtyZip('https://s3.amazonaws.com/tripdata/202101-citibike-tripdata.csv.zip')

legacyTrips['start station id'] = legacyTrips['start station id'].map(stationNameDictionary)
legacyTrips['end station id'] = legacyTrips['end station id'].map(stationNameDictionary)

legacyTrips.head()

,tripduration,starttime,stoptime,start station id,start station name,start station latitude,start station longitude,end station id,end station name,end station latitude,end station longitude,bikeid,usertype,birth year,gender
0,2513,2021-01-01 00:00:11.9020,2021-01-01 00:42:05.2260,4042.08,Underhill Ave & Lincoln Pl,40.674012,-73.967146,4042.08,Underhill Ave & Lincoln Pl,40.674012,-73.967146,47812,Customer,1969,0
1,2519,2021-01-01 00:00:15.0960,2021-01-01 00:42:14.9780,4042.08,Underhill Ave & Lincoln Pl,40.674012,-73.967146,4042.08,Underhill Ave & Lincoln Pl,40.674012,-73.967146,47571,Customer,1969,0
2,1207,2021-01-01 00:00:28.9300,2021-01-01 00:20:36.6510,7188.10,E 81 St & Park Ave,40.776777,-73.959010,6912.01,7 Ave & Central Park South,40.766741,-73.979069,37451,Subscriber,2002,1
3,2506,2021-01-01 00:00:32.7130,2021-01-01 00:42:19.3980,4042.08,Underhill Ave & Lincoln Pl,40.674012,-73.967146,4042.08,Underhill Ave & Lincoln Pl,40.674012,-73.967146,48884,Customer,2002,1
4,959,2021-01-01 00:00:35.3650,2021-01-01 00:16:34.6010,NaN,Water - Whitehall Plaza,40.702551,-74.012723,5181.04,Cherry St,40.712199,-73.979481,26837,Customer,2002,1


Looks good. In fact, looks great, because station IDs that are not in the dictionary of current stations are replaced with NaN. We can use the PANDAS dropna fuction to remove them... later. First we will do a little more processing and cleaning.

Some column names have changed in the new era. Spaces have been replaced with underscores in the new data, and the time stamp column names are prepositional phrases.

We are only going to be using trip start times, end times, and the station IDs for the starting stations and end stations, so these are the only ones we will bother to rename.

In [119]:
legacyColumnRename = dict({'starttime': 'started_at', 'stoptime': 'ended_at', 'start station id': 'start_station_id', 'end station id': 'end_station_id'})
legacyTrips.rename(columns=legacyColumnRename, inplace=True)

Then we can use the column renaming dictionary to cull the unwanted columns from our DataFrame.

In [120]:
legacyTrips = legacyTrips[legacyColumnRename.values()]
legacyTrips.head()

,started_at,ended_at,start_station_id,end_station_id
0,2021-01-01 00:00:11.9020,2021-01-01 00:42:05.2260,4042.08,4042.08
1,2021-01-01 00:00:15.0960,2021-01-01 00:42:14.9780,4042.08,4042.08
2,2021-01-01 00:00:28.9300,2021-01-01 00:20:36.6510,7188.10,6912.01
3,2021-01-01 00:00:32.7130,2021-01-01 00:42:19.3980,4042.08,4042.08
4,2021-01-01 00:00:35.3650,2021-01-01 00:16:34.6010,NaN,5181.04


And finally drop NaNs. 

Had we done this before trimming the columns we might have lost desired data if there was mssing information in columns that we aren't even going to be using.

In [121]:
legacyTrips.dropna(inplace=True)
legacyTrips.isna().sum()

started_at          0
ended_at            0
start_station_id    0
end_station_id      0
dtype: int64

We can wrap this all into a function that accepts a URL of an S3 resource, checks whether any formatting is needed, performs the changes, and returns a cleaned and formatted DataFrame.

In [122]:
def legacyCheckFix(s3URL):
  legacyTrips = readDirtyZip(s3URL)

  if 'start station id' in legacyTrips.columns:
    legacyTrips['start station id'] = legacyTrips['start station id'].map(stationNameDictionary)
    legacyTrips['end station id'] = legacyTrips['end station id'].map(stationNameDictionary)

    legacyTrips.rename(columns=legacyColumnRename, inplace=True)
  
  legacyTrips = legacyTrips[legacyColumnRename.values()]
  legacyTrips.dropna(inplace=True)
  return legacyTrips

# The S3 Bucket

We don't really want to go and find the URLs manually, so maybe a look at the contents of the bucket is in order. We will use Boto3 to do this, connecting to S3 without a signature to avoid having to configure anything.

In [123]:
%%capture
!pip install boto3
import boto3
from botocore import UNSIGNED
from botocore.client import Config

In [124]:
s3 = boto3.client('s3', config=Config(signature_version=UNSIGNED))
s3.list_objects(Bucket='tripdata')['Contents'][0:3]

[{'ETag': '"b520a12de58eea58a3586f89bfcfbd9d-2"',
  'Key': '201306-citibike-tripdata.zip',
  'LastModified': datetime.datetime(2018, 4, 30, 13, 18, 55, tzinfo=tzlocal()),
  'Size': 16785103,
  'StorageClass': 'STANDARD'},
 {'ETag': '"7b3b260b2ab2e5349320121d04bd821c-22"',
  'Key': '201307-201402-citibike-tripdata.zip',
  'LastModified': datetime.datetime(2017, 1, 18, 22, 23, 25, tzinfo=tzlocal()),
  'Size': 178262576,
  'StorageClass': 'STANDARD'},
 {'ETag': '"dd3e6fd5f91715b31eae72868086c08c-4"',
  'Key': '201307-citibike-tripdata.zip',
  'LastModified': datetime.datetime(2017, 1, 18, 22, 23, 27, tzinfo=tzlocal()),
  'Size': 27074629,
  'StorageClass': 'STANDARD'}]

It looks like the 'LastModified' values aren't reliably correlated with the time period covered by the collected trip data.

The 'Key' key, which returns the name of a zip directory, is what we want.

If we provide a starting month and year and an ending month, we can get a list of all the URLs that correspond to the published trip data for that time span.

Instead of a function, this time a class makes more sense. Included in the class is a filename generator, more on that later.

In [144]:
class fileListUrls:

  def __init__(self, startMonth, startYear, endMonth, endYear):
    self.startMonth = startMonth
    self.startYear = startYear
    self.endMonth = endMonth
    self.endYear = endYear

  def tripURLs(self):
    tripURLs = []
    monthRange = pd.date_range((str(self.startYear) + '-' + str(self.startMonth)), (str(self.endYear) + '-' + str(self.endMonth)) , freq='MS').strftime("%Y%m").tolist()
    for dictName in s3.list_objects(Bucket='tripdata')['Contents']:
      for month in monthRange:
        if dictName['Key'].startswith(month):
          tripURLs.append('https://s3.amazonaws.com/tripdata/' + dictName['Key'])
          monthRange.remove(month)
  
    tripURLs.reverse()
    return tripURLs

  def nameForCsv(self):
    nameForCsv = '/drive/MyDrive/' + str(self.startYear) + str(self.startMonth).zfill(2) + '-' + str(self.endYear) + str(self.endMonth).zfill(2) + 'csv.xz'
    return nameForCsv

Putting it all together, if we provide a starting month and year, an ending month and year, we can grab all of the trip data in that range, clean it, and merge it into one large dataset.

In order to be able to run this in Colab without worrying about running out of resources, we will write to disk after each month is processed, appending to the CSV created after the first pass.

In this example I'm writing to the base directory in my Google Drive.

In [ ]:
from google.colab import drive
drive.mount('/drive')

In [145]:
urlRange = fileListUrls(input("Start month (integer): "), input("Start year: "), input("End month (integer, inclusive): "), input("End year:"))

csvUrls = urlRange.tripURLs()

hotTrips = legacyCheckFix(csvUrls[0])

print("Writing first CSV...")

hotTrips.to_csv(urlRange.nameForCsv())

del hotTrips

for url in csvUrls[1:]:
  hotTrips = legacyCheckFix(url)
  print('Appending...')
  hotTrips.to_csv(urlRange.nameForCsv(), mode='a', header=False)
  del hotTrips

Start month (integer): 2
Start year: 2020
End month (integer, inclusive): 4
End year:2020
Writing first CSV...
Appending...
Appending...


In [146]:
checkTrips = pd.read_csv('/drive/MyDrive/202002-202004csv.xz')

,Unnamed: 0,started_at,ended_at,start_station_id,end_station_id
0,0,2020-04-01 00:00:15.2800,2020-04-01 00:09:09.8730,5553.10,5929.01
1,1,2020-04-01 00:02:28.9430,2020-04-01 00:11:18.3410,7079.06,7520.07
2,2,2020-04-01 00:02:56.0900,2020-04-01 00:27:44.2530,7175.05,6893.10
3,3,2020-04-01 00:03:25.1410,2020-04-01 00:09:06.3190,4366.01,4568.01
4,4,2020-04-01 00:05:35.2790,2020-04-01 00:42:21.9080,4526.01,4617.01
...,...,...,...,...,...
2529211,1146824,2020-02-29 23:58:43.6800,2020-03-01 00:02:07.5940,5445.07,5584.04
2529212,1146825,2020-02-29 23:58:48.0230,2020-03-01 00:02:11.6640,5445.07,5584.04
2529213,1146826,2020-02-29 23:58:52.5640,2020-03-01 00:04:50.5370,6206.08,5938.11
2529214,1146827,2020-02-29 23:58:54.5820,2020-03-01 00:01:43.7770,4237.01,4425.02
